In [1]:
import os
import cv2
import time
import json
import pandas as pd
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import warnings
warnings.filterwarnings('ignore')

2024-05-31 07:37:52.108968: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-31 07:37:52.145463: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 07:37:52.917288: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def makedir(directory): # Función para cargar los datos
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory


def save_plot(path, save_path=None, extra_img_path=None): # Función para guardar una imagen con cada clase
    classes = sorted(os.listdir(path))
    fig, axs = plt.subplots(5, 4, figsize=(20, 20))
    plt.rcParams.update({'font.size': 22})
    for i, c in enumerate(classes):
        img = mpimg.imread(path + c + '/' + os.listdir(path + c)[0])
        axs[i//4, i%4].imshow(img)
        axs[i//4, i%4].set_title(c)
        axs[i//4, i%4].axis('off')
    axs[4, 3].imshow(mpimg.imread(extra_img_path))
    axs[4, 3].axis('off')
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.show()

In [3]:
def draw_hand_landmarks(frame, landmarks, margin=15): # Función para dibujar los puntos de referencia de la mano
    height, width, _ = frame.shape  # Obtiene las dimensiones del frame
    x_min, y_min, x_max, y_max = width, height, 0, 0  # Inicializa los límites del rectángulo que rodea la mano

    for i, point in enumerate(landmarks.landmark):
        x, y = int(point.x * width), int(point.y * height)  # Calcula las coordenadas del punto en píxeles
        color = (0, 255, 0) if i % 4 == 0 else (0, 0, 255)  # Alterna el color entre verde y rojo para cada punto
        cv2.circle(frame, (x, y), int(height * 0.01), color, -1)  # Dibuja un círculo en el punto

        # Actualiza las coordenadas mínimas y máximas para dibujar un rectángulo alrededor de la mano
        x_min = min(x_min, x)
        y_min = min(y_min, y)
        x_max = max(x_max, x)
        y_max = max(y_max, y)

    # Definir conexiones solo para el contorno de la mano y los dedos
    connections = [[1, 2], [2, 3], [3, 4],  # Dedo pulgar
                   [5, 6], [6, 7], [7, 8],  # Dedo índice
                   [9, 10], [10, 11], [11, 12],  # Dedo medio
                   [13, 14], [14, 15], [15, 16],  # Dedo anular
                   [17, 18], [18, 19], [19, 20],  # Dedo meñique
                   [0, 1],[0, 17],[1, 5], [5, 9], [9, 13], [13, 17]]  # Conexiones de la palma de la mano

    # Dibujar las conexiones entre los puntos de referencia de la mano
    for connection in connections:
        cv2.line(frame, (int(landmarks.landmark[connection[0]].x * width), int(landmarks.landmark[connection[0]].y * height)),
                         (int(landmarks.landmark[connection[1]].x * width), int(landmarks.landmark[connection[1]].y * height)), (0, 0, 255), 2)

    hand_width = x_max - x_min
    hand_height = y_max - y_min

    # Calculate the side length of the square bounding box
    square_side = max(hand_width, hand_height)

    # Calculate the coordinates of the square bounding box
    x_min_square = x_min + hand_width // 2 - square_side // 2
    x_max_square = x_min_square + square_side
    y_min_square = y_min + hand_height // 2 - square_side // 2
    y_max_square = y_min_square + square_side

    # Añadir un margen al cuadrado
    x_min_square = max(0, x_min_square)
    y_min_square = max(0, y_min_square)
    x_max_square = min(width, x_max_square)
    y_max_square = min(height, y_max_square)

    # Dibujar el cuadrado que rodea la mano
    cv2.rectangle(frame, (x_min_square-margin, y_min_square-margin), (x_max_square+margin, y_max_square+margin), (0, 0, 255), 2)

    # Return the square bounding box instead of the hand rectangle
    return frame, (x_min_square-margin, y_min_square-margin, x_max_square+margin, y_max_square+margin)




In [6]:
def capture_frames(hands, classes, path, num_frames=200):
    classes_idx = {c: i for i, c in enumerate(classes)}

    cap = cv2.VideoCapture(0)  # Inicializa la cámara

    makedir(path)  # Crea la carpeta 'data' si no existe

    for c in classes:
        makedir(path + c)
    letter_idx = 0  # Índice de la letra actual
    frames_taken = 0  # Contador para el número de frames tomados por letra

    frames_dataframe = []

    while True:
        ret, frame = cap.read()  # Lee un frame de la cámara

        # Si no se pudo leer el frame, termina el ciclo
        if not ret:
            print("Error al leer el frame")
            break

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convierte el frame de BGR a RGB

        hands_results = hands.process(img)  # Procesa el frame con el modelo de manos

        # Dibuja puntos y líneas para las manos en la imagen negra
        if hands_results.multi_hand_landmarks:
            for landmarks in hands_results.multi_hand_landmarks:
                drawn_frame, hand_rect = draw_hand_landmarks(frame, landmarks)  # Dibuja puntos y líneas para las mano
                
                if frames_taken < num_frames:
                    letter = classes[letter_idx]  # Obtiene la letra actual

                    # recortar la mano y guardarla 
                    try:
                        reshaped_frame = drawn_frame[hand_rect[1]:hand_rect[3], hand_rect[0]:hand_rect[2]] # Recorta el cuadrado que rodea la mano y lo redimensiona
                        reshaped_frame = cv2.cvtColor(reshaped_frame, cv2.COLOR_BGR2GRAY)  # Convierte la imagen a escala de grises
                        reshaped_frame = cv2.resize(reshaped_frame, (28, 28), interpolation=cv2.INTER_AREA) # Redimensiona la imagen a 28x28 píxel  es
                        
                        # guardar el frame en el dataframe
                        frames_dataframe.append([classes_idx[letter]] + reshaped_frame.flatten().tolist())

                        # reshaped_frame = reshaped_frame.reshape(1, 28, 28,  1)
                        frames_taken += 1  # Incrementa el contador de frames tomados

                    except Exception as e:
                        print(e)
                        pass
                            
                if frames_taken >= num_frames:
                    time.sleep(3)
                    letter_idx += 1
                    frames_taken = 0
                if letter_idx >= len(classes):
                    pd.DataFrame(frames_dataframe, columns=['label'] + ['pixel'+str(i) for i in range(len(frames_dataframe[0])-1)]).to_csv(f'{path}/frames{time.strftime("%Y%m%d%H%M%S")}.csv', index=False)
                    break
            
            cv2.putText(drawn_frame, letter + ' ' + str(frames_taken), (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imshow('Pose Detection', frame)  # Muestra el frame con las manos detectadas
        
        # Si se presiona la tecla 'q', termina el ciclo
        if (cv2.waitKey(1) & 0xFF == ord('q')) or letter_idx >= len(classes):
            break

    cap.release()  # Libera la cámara
    cv2.destroyAllWindows() # Cerrar todas las ventanas
    hands.close() # Cerrar el modelo de manos


In [18]:
mp_hands = mp.solutions.hands # Importar el modelo de manos de MediaPipe
hands = mp_hands.Hands(max_num_hands=1) # Inicializar el modelo de manos

# Ruta de la carpeta donde se guardarán las imágenes en formato csv
path_db = f'data/backups/'
classes = list(json.load(open(f'{path_db}classes.json')).keys())
print(classes)
# capture_frames(hands, classes, path_db) # Capturar frames de la cámara y guardarlos en carpetas correspondientes a cada letra



['A', 'B', 'C', 'D', 'E', 'F', 'G', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U']


I0000 00:00:1717160589.746552  361031 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1717160589.747662  374274 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.6-manjaro1.1), renderer: Mesa Intel(R) Xe Graphics (TGL GT2)
